#### 0. LangChain에서 도구(tool) 활용 방법

In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

gs
z6


In [2]:
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(
#     base_url="https://api.groq.com/openai/v1",
#     #model="meta-llama/llama-4-scout-17b-16e-instruct",
#     model="moonshotai/kimi-k2-instruct-0905",
#     temperature=0
# )

from langchain_upstage import ChatUpstage
llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
    )
print(llm)

c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-SBe-Yh6W-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


client=<openai.resources.chat.completions.completions.Completions object at 0x000002B14FDBC200> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002B14FE29550> model_name='solar-pro' temperature=0.5 model_kwargs={} upstage_api_key=SecretStr('**********') upstage_api_base='https://api.upstage.ai/v1'


- [tool decorator](https://python.langchain.com/docs/how_to/custom_tools/#tool-decorator)를 사용하면 쉽게 도구를 만들 수 있습니다

In [3]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """숫자 a와 b를 더합니다."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """숫자 a와 b를 곱합니다."""
    return a * b

In [5]:
# @tool 데코레이터로 선언된 함수는 직접 호출할 수 없다.
# add(10,20)


- LLM을 호출했을 때와 도구를 사용했을 때의 차이를 알아봅니다

In [ ]:
query = '3 곱하기 5는?'
llm_result = llm.invoke(query)


NameError: name 'llm_reuslt' is not defined

- 도구 리스트는 LLM에 해당하는 `BaseModel` 클래스에 `bind_tools` 메서드를 통해 전달합니다

In [10]:
llm_with_tools = llm.bind_tools([add, multiply])

print(type(llm_with_tools))
print(llm_with_tools)

<class 'langchain_core.runnables.base.RunnableBinding'>
bound=ChatUpstage(client=<openai.resources.chat.completions.completions.Completions object at 0x000002B14FDBC200>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002B14FE29550>, model_name='solar-pro', temperature=0.5, model_kwargs={}, upstage_api_key=SecretStr('**********'), upstage_api_base='https://api.upstage.ai/v1') kwargs={'tools': [{'type': 'function', 'function': {'name': 'add', 'description': '숫자 a와 b를 더합니다.', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'multiply', 'description': '숫자 a와 b를 곱합니다.', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}]} config={} config_factories=[]


- `AIMessage`의 `additional_kwargs` 속성은 `tool_calls`를 포함합니다
- `tool_calls`는 도구를 호출하는 메시지를 포함합니다

In [11]:
tool_result = llm_with_tools.invoke(query)

tool_result

AIMessage(content='[질문 "3 곱하기 5는?"을 해결하기 위해 곱셈 연산이 필수적이며, `multiply` 함수가 직접 필요한 계산을 수행하므로 호출합니다.]  \n\n결과: 15', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-a4b5ec44f7fe4426a909aff8d75fd16b', 'function': {'arguments': '{"a": 3, "b": 5}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 578, 'total_tokens': 634, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': '241af6de-6f73-49fe-98d0-f6b99c214266', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--c3d8c16b-a319-4c2e-b92e-93e5bf04ef0e-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 5}, 'id': 'chatcmpl-tool-a4b5ec44f7fe4426a909aff8d75fd16b', 'type': 'tool_call'}], usage_metada

In [15]:
tool_result.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 5},
  'id': 'chatcmpl-tool-a4b5ec44f7fe4426a909aff8d75fd16b',
  'type': 'tool_call'}]

In [20]:
from typing import Sequence

from langchain_core.messages import AnyMessage, HumanMessage

query="20과 10을 더하고 곱한 결과"
human_message = HumanMessage(query)
message_list: Sequence[AnyMessage] = [human_message] 


- `tool_calls` 속성은 도구를 호출하는 메시지를 포함합니다
- `tool_calls`를 가진 `AIMessage`의 형태를 기억하기

In [21]:
ai_message = llm_with_tools.invoke(message_list)
ai_message

AIMessage(content='[질문에서 "20과 10을 더하고 곱한 결과"를 요구하므로, 덧셈(add)과 곱셈(multiply) 연산이 모두 필수적입니다. 두 함수 호출은 각각 필요한 계산을 수행하기 위해 선택되었습니다.]  \n\n결과:  \n- 덧셈: 20 + 10 = **30**  \n- 곱셈: 20 × 10 = **200**', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-fb98cee10885441db7be0f6950d66c53', 'function': {'arguments': '{"a": 20, "b": 10}', 'name': 'add'}, 'type': 'function'}, {'id': 'chatcmpl-tool-3a43f29e9ad5498ea3ad64dd587cfbba', 'function': {'arguments': '{"a": 20, "b": 10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 111, 'prompt_tokens': 582, 'total_tokens': 693, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': '8c9080b2-7c47-4771-918a-1f4286f5bca2', 'service_tier': None, 'finish_reason': 'tool_ca

In [22]:
ai_message.tool_calls

[{'name': 'add',
  'args': {'a': 20, 'b': 10},
  'id': 'chatcmpl-tool-fb98cee10885441db7be0f6950d66c53',
  'type': 'tool_call'},
 {'name': 'multiply',
  'args': {'a': 20, 'b': 10},
  'id': 'chatcmpl-tool-3a43f29e9ad5498ea3ad64dd587cfbba',
  'type': 'tool_call'}]

In [23]:
from pprint import pprint

message_list.append(ai_message)

pprint(message_list)

[HumanMessage(content='20과 10을 더하고 곱한 결과', additional_kwargs={}, response_metadata={}),
 AIMessage(content='[질문에서 "20과 10을 더하고 곱한 결과"를 요구하므로, 덧셈(add)과 곱셈(multiply) 연산이 모두 필수적입니다. 두 함수 호출은 각각 필요한 계산을 수행하기 위해 선택되었습니다.]  \n\n결과:  \n- 덧셈: 20 + 10 = **30**  \n- 곱셈: 20 × 10 = **200**', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-fb98cee10885441db7be0f6950d66c53', 'function': {'arguments': '{"a": 20, "b": 10}', 'name': 'add'}, 'type': 'function'}, {'id': 'chatcmpl-tool-3a43f29e9ad5498ea3ad64dd587cfbba', 'function': {'arguments': '{"a": 20, "b": 10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 111, 'prompt_tokens': 582, 'total_tokens': 693, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id'

- `AIMessage`의 `tool_calls`를 활용해서 도구를 직접 호출할 수도 있습니다

In [27]:
ai_message.tool_calls

[{'name': 'add',
  'args': {'a': 20, 'b': 10},
  'id': 'chatcmpl-tool-fb98cee10885441db7be0f6950d66c53',
  'type': 'tool_call'},
 {'name': 'multiply',
  'args': {'a': 20, 'b': 10},
  'id': 'chatcmpl-tool-3a43f29e9ad5498ea3ad64dd587cfbba',
  'type': 'tool_call'}]

In [49]:
tool_message = multiply.invoke(ai_message.tool_calls[0])

print(tool_message)

content='15' name='multiply' tool_call_id='chatcmpl-tool-638066a86a0143c682396fe4fde379e5'


- 하지만 에이전트의 경우 도구를 직접 호출하는 것이 아니라 도구를 호출하는 메시지를 만들어서 전달합니다

In [50]:
message_list.append(tool_message)

pprint(message_list)

[HumanMessage(content='3 곱하기 5는?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='[주어진 질문은 "3 곱하기 5"의 결과를 묻는 것으로, 이는 직접적으로 곱셈 연산이 필요합니다. 따라서 \'multiply\' 함수를 호출하는 것이 필수적입니다. 다른 방법으로는 일반 지식으로도 답할 수 있지만, 규칙에 따라 함수 사용이 필요한 경우에만 호출해야 하며, 이 경우 곱셈 연산이 질문의 핵심이므로 함수 호출이 정당합니다.] \n\n정답: 15', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-638066a86a0143c682396fe4fde379e5', 'function': {'arguments': '{"a": 3, "b": 5}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 578, 'total_tokens': 667, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': '11c36d9a-83e6-406f-b0a9-0ee6e9525b33', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--1

In [51]:
tool_result = llm_with_tools.invoke(message_list)

pprint(tool_result)

AIMessage(content='3 곱하기 5는 15입니다. \n\n함수 호출 결과와 일치하는 정답을 확인했습니다.\n\n[질문의 핵심인 "3 곱하기 5" 연산을 수행하기 위해 \'multiply\' 함수가 필수적으로 요구됩니다. 일반 지식으로도 답변 가능하지만, 문제 해결을 위해 함수 사용이 명시적으로 요청된 상황으로 판단됩니다.] \n\n최종 답변: 15', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-99c188eebb814a838f6526d0c3606084', 'function': {'arguments': '{"a": 3, "b": 5}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 682, 'total_tokens': 768, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': '37df693b-c96e-440c-8257-b928d4e05a01', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--bb7bb29f-23f3-4d89-aecd-5884b8a4ccb9-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 5}

- `message_list`의 순서를 기억하기

In [52]:
message_list

[HumanMessage(content='3 곱하기 5는?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='[주어진 질문은 "3 곱하기 5"의 결과를 묻는 것으로, 이는 직접적으로 곱셈 연산이 필요합니다. 따라서 \'multiply\' 함수를 호출하는 것이 필수적입니다. 다른 방법으로는 일반 지식으로도 답할 수 있지만, 규칙에 따라 함수 사용이 필요한 경우에만 호출해야 하며, 이 경우 곱셈 연산이 질문의 핵심이므로 함수 호출이 정당합니다.] \n\n정답: 15', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-638066a86a0143c682396fe4fde379e5', 'function': {'arguments': '{"a": 3, "b": 5}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 578, 'total_tokens': 667, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': '11c36d9a-83e6-406f-b0a9-0ee6e9525b33', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--1